!pip install python-box timm pytorch-lightning==1.4.0 grad-cam ttach

In [1]:
%%python
    
import os
import sys


import warnings
from pprint import pprint
from glob import glob
from tqdm import tqdm
import pickle

from sklearn.linear_model import Ridge
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as T
sys.path.append('../input/pythonbox')
from box import Box
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
#from pytorch_grad_cam import GradCAMPlusPlus
#from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
from fastai.vision.all import *

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule

sys.path.append('../input/poolformer-master')
import models as PoolFormerModels #poolformer

print(pl.__version__)
warnings.filterwarnings("ignore")



config_ensemble = {'exp_name':'exp073',
            'seed': 2021,
              'n_splits': 5,
              'oof_expname':{
                  1:'exp049',
                  2:'exp054',
                  3:'exp060',
                  4:'exp065',
                  5:'exp069',
                  6:'exp074',
              },
              'model':'Ridge'
}

config_ensemble = Box(config_ensemble)

#049
config_exp049 = {'exp_name':'exp049',
            'seed': 2021,
          'root': '/kaggle/input/petfinder-pawpularity-score/', 
          'n_splits': 5,
          'epoch': 20,
          'transform':{
              'name': 'get_default_transforms',
              'image_size': 384
          },
          'test_loader': {
              'batch_size': 32,
              'shuffle': False,
              'num_workers': os.cpu_count(),
              'pin_memory': False,
              'drop_last': False
          },
          'model':{
              'name': 'swin_large_patch4_window12_384',
              'output_dim': 1
          },
          'loss': 'nn.BCEWithLogitsLoss',
}

config_exp049 = Box(config_exp049)

#054
config_exp054 = {'exp_name':'exp054',
          'seed': 2021,
          'root': '/kaggle/input/petfinder-pawpularity-score/', 
          'n_splits': 5,
          'transform':{
              'name': 'get_default_transforms',
              'image_size': 224
          },
          'test_loader': {
              'batch_size': 32,
              'shuffle': False,
              'num_workers': os.cpu_count(),
              'pin_memory': False,
              'drop_last': False
          },
          'model':{
              'name': 'swin_large_patch4_window7_224',
              'output_dim': 1
          },
          'optimizer':{
              'name': 'optim.AdamW',
              'params':{
                  'lr': 1e-5
              },
          },
          'scheduler':{
              'name': 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
              'params':{
                  'T_0': 20,
                  'eta_min': 1e-4,
              }
          },
          'loss': 'nn.BCEWithLogitsLoss',
}

config_exp054 = Box(config_exp054)

#060
config_exp060 = {'exp_name':'exp060',
          'seed': 2021,
          'root': '/kaggle/input/petfinder-pawpularity-score/', 
          'n_splits': 5,
          'transform':{
              'name': 'get_default_transforms',
              'image_size': 224
          },
          'test_loader': {
              'batch_size': 32,
              'shuffle': False,
              'num_workers': os.cpu_count(),
              'pin_memory': False,
              'drop_last': False
          },
          'model':{
              'name': 'swin_large_patch4_window7_224_in22k',
              'output_dim': 1
          },
          'optimizer':{
              'name': 'optim.AdamW',
              'params':{
                  'lr': 1e-5
              },
          },
          'scheduler':{
              'name': 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
              'params':{
                  'T_0': 20,
                  'eta_min': 1e-4,
              }
          },
          'loss': 'nn.BCEWithLogitsLoss',
}

config_exp060 = Box(config_exp060)

#065
config_exp065 = {'exp_name':'exp065',
          'seed': 2021,
          'root': '/kaggle/input/petfinder-pawpularity-score/', 
          'n_splits': 5,
          'transform':{
              'name': 'get_default_transforms',
              'image_size': 384
          },
          'test_loader': {
              'batch_size': 32,
              'shuffle': False,
              'num_workers': os.cpu_count(),
              'pin_memory': False,
              'drop_last': False
          },
          'model':{
              'name': 'swin_large_patch4_window12_384',
              'output_dim': 1
          },
          'optimizer':{
              'name': 'optim.AdamW',
              'params':{
                  'lr': 1e-5
              },
          },
          'scheduler':{
              'name': 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
              'params':{
                  'T_0': 20,
                  'eta_min': 1e-4,
              }
          },
          'loss': 'nn.BCEWithLogitsLoss',
}

config_exp065 = Box(config_exp065)

#069
config_exp069 = {'exp_name':'exp069',
          'seed': 2021,
          'root': '/kaggle/input/petfinder-pawpularity-score/', 
          'n_splits': 5,
          'transform':{
              'name': 'get_default_transforms',
              'image_size': 224
          },
          'test_loader': {
              'batch_size': 32,
              'shuffle': False,
              'num_workers': os.cpu_count(),
              'pin_memory': False,
              'drop_last': False
          },
          'model':{
              'name': 'swin_large_patch4_window7_224',
              'output_dim': 1
          },
          'optimizer':{
              'name': 'optim.AdamW',
              'params':{
                  'lr': 1e-5
              },
          },
          'scheduler':{
              'name': 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
              'params':{
                  'T_0': 20,
                  'eta_min': 1e-4,
              }
          },
          'loss': 'nn.BCEWithLogitsLoss',
}

config_exp069 = Box(config_exp069)

#074
config_exp074 = {'exp_name':'exp074',
          'seed': 2021,
          'root': '/kaggle/input/petfinder-pawpularity-score/', 
          'n_splits': 5,
          'transform':{
              'name': 'get_default_transforms',
              'image_size': 224
          },
          'test_loader': {
              'batch_size': 32,
              'shuffle': False,
              'num_workers': os.cpu_count(),
              'pin_memory': False,
              'drop_last': False
          },
          'model':{
              'name': 'tf_efficientnetv2_b1',
              'output_dim': 1
          },
          'optimizer':{
              'name': 'optim.AdamW',
              'params':{
                  'lr': 1e-5
              },
          },
          'scheduler':{
              'name': 'optim.lr_scheduler.CosineAnnealingWarmRestarts',
              'params':{
                  'T_0': 20,
                  'eta_min': 1e-4,
              }
          },
          'loss': 'nn.BCEWithLogitsLoss',
}

config_exp074 = Box(config_exp074)

class PetfinderDataset_exp049(Dataset):
    def __init__(self, df, image_size=224):
        self._X = df["Id"].values
        self._y = None
        if "Pawpularity" in df.keys():
            self._y = df["Pawpularity"].values
        self._transform = T.Compose([
                                        T.Resize(image_size),  # 1
                                        T.CenterCrop([image_size, image_size]),  # 2
                                    ]
                                    )

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)
        if self._y is not None:
            label = self._y[idx]
            return image, label
        return image

class PetfinderDataset_exp054(Dataset):
    def __init__(self, df, image_size=224):
        dense_feature_cols = [
            'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
            'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur',
            ]
        self._X = df["Id"].values
        self.dense_features = df[dense_feature_cols].values
        self._y = None
        if "Pawpularity" in df.keys():
            self._y = df["Pawpularity"].values
        self._transform = T.Compose([
                                        T.Resize(image_size),  # 1
                                        T.CenterCrop([image_size, image_size]),  # 2
                                    ]
                                    )

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)

        features = self.dense_features[idx, :]
        if self._y is not None:
            label = self._y[idx]
            return image, features, label
        return image, features

class PetfinderDataset_exp060(Dataset):
    def __init__(self, df, image_size=224):
        dense_feature_cols = [
            'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
            'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur',
            'height', 'width', 'aspect'
            ]
        self._X = df["Id"].values
        self.dense_features = df[dense_feature_cols].values
        self._y = None
        if "Pawpularity" in df.keys():
            self._y = df["Pawpularity"].values
        self._transform = T.Compose([
                                        T.Resize(image_size),  # 1
                                        T.CenterCrop([image_size, image_size]),  # 2
                                    ]
                                    )

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)

        features = self.dense_features[idx, :]
        if self._y is not None:
            label = self._y[idx]
            return image, features, label
        return image, features    

class PetfinderDataset_exp065(Dataset):
    def __init__(self, df, image_size=224):
        self._X = df["Id"].values
        self._y = None
        if "Pawpularity" in df.keys():
            self._y = df["Pawpularity"].values
        self._transform = T.Compose([
                                        T.Resize(image_size),  # 1
                                        T.CenterCrop([image_size, image_size]),  # 2
                                    ]
                                    )

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)
        if self._y is not None:
            label = self._y[idx]
            return image, label
        return image

class PetfinderDataset_exp069(Dataset):
    def __init__(self, df, image_size=224):
        dense_feature_cols = [
            'height', 'width', 'aspect'
            ]
        self._X = df["Id"].values
        self.dense_features = df[dense_feature_cols].values
        self._y = None
        if "Pawpularity" in df.keys():
            self._y = df["Pawpularity"].values
        self._transform = T.Compose([
                                        T.Resize(image_size),  # 1
                                        T.CenterCrop([image_size, image_size]),  # 2
                                    ]
                                    )

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)

        features = self.dense_features[idx, :]
        if self._y is not None:
            label = self._y[idx]
            return image, features, label
        return image, features

class PetfinderDataset_exp074(Dataset):
    def __init__(self, df, image_size=224):
        dense_feature_cols = [
            'Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
            'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur',
            'height', 'width', 'aspect'
            ]
        self._X = df["Id"].values
        self.dense_features = df[dense_feature_cols].values
        self._y = None
        if "Pawpularity" in df.keys():
            self._y = df["Pawpularity"].values
        self._transform = T.Compose([
                                        T.Resize(image_size),  # 1
                                        T.CenterCrop([image_size, image_size]),  # 2
                                    ]
                                    )

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)

        features = self.dense_features[idx, :]
        if self._y is not None:
            label = self._y[idx]
            return image, features, label
        return image, features
    
class PetfinderDataModule(LightningDataModule):
    def __init__(
        self,
        test_df,
        cfg,
    ):
        super().__init__()
        self._test_df = test_df
        self._cfg = cfg

    def __create_dataset(self, train=True):
        
        if self._cfg.exp_name=='exp049':
            return (
                PetfinderDataset_exp049(self._test_df, self._cfg.transform.image_size)
                if train
                else PetfinderDataset_exp049(self._test_df, self._cfg.transform.image_size)
            )
        if self._cfg.exp_name=='exp054':
            return (
                PetfinderDataset_exp054(self._test_df, self._cfg.transform.image_size)
                if train
                else PetfinderDataset_exp054(self._test_df, self._cfg.transform.image_size)
            )
        if self._cfg.exp_name=='exp060':
            return (
                PetfinderDataset_exp060(self._test_df, self._cfg.transform.image_size)
                if train
                else PetfinderDataset_exp060(self._test_df, self._cfg.transform.image_size)
            )
        if self._cfg.exp_name=='exp065':
            return (
                PetfinderDataset_exp065(self._test_df, self._cfg.transform.image_size)
                if train
                else PetfinderDataset_exp065(self._test_df, self._cfg.transform.image_size)
            )
        if self._cfg.exp_name=='exp069':
            return (
                PetfinderDataset_exp069(self._test_df, self._cfg.transform.image_size)
                if train
                else PetfinderDataset_exp069(self._test_df, self._cfg.transform.image_size)
            )
        if self._cfg.exp_name=='exp074':
            return (
                PetfinderDataset_exp074(self._test_df, self._cfg.transform.image_size)
                if train
                else PetfinderDataset_exp074(self._test_df, self._cfg.transform.image_size)
            )

    def test_dataloader(self):
        dataset = self.__create_dataset(True)
        return DataLoader(dataset, **self._cfg.test_loader)
    
IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB

def get_default_transforms():
    transform = {
        "train": T.Compose(
            [
                T.RandomHorizontalFlip(),
                T.RandomVerticalFlip(),
                T.RandomAffine(15, translate=(0.1, 0.1), scale=(0.9, 1.1)),
                T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1),
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
        "val": T.Compose(
            [
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
    }
    return transform

def mixup(x: torch.Tensor, y: torch.Tensor, alpha: float = 1.0):
    assert alpha > 0, "alpha should be larger than 0"
    assert x.size(0) > 1, "Mixup cannot be applied to a single instance."

    lam = np.random.beta(alpha, alpha)
    rand_index = torch.randperm(x.size()[0])
    mixed_x = lam * x + (1 - lam) * x[rand_index, :]
    target_a, target_b = y, y[rand_index]
    return mixed_x, target_a, target_b, lam
    
class Model_exp049(pl.LightningModule):
    def __init__(self, cfg, val_losses=None):
        super().__init__()
        self.cfg = cfg
        self.__build_model()
        self._criterion = eval(self.cfg.loss)()
        self.transform = get_default_transforms()
        self.save_hyperparameters(cfg)
        self.val_losses = val_losses 

    def __build_model(self):
        self.backbone = create_model(
            self.cfg.model.name, pretrained=False, num_classes=0, in_chans=3
        )
        num_features = self.backbone.num_features
        self.fc = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(num_features, self.cfg.model.output_dim)
        )

    def forward(self, x):
        f = self.backbone(x)
        out = self.fc(f)
        return out

    def training_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'train')
        self.log("train/loss", loss)
        return {'loss': loss, 'pred': pred, 'labels': labels}
        
    def validation_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'val')
        self.log("val/loss", loss)
        return {'pred': pred, 'labels': labels}
    
    def __share_step(self, batch, mode):
        images, labels = batch
        labels = labels.float() / 100.0
        images = self.transform[mode](images)
        
        if torch.rand(1)[0] < 0.5 and mode == 'train':
            mix_images, target_a, target_b, lam = mixup(images, labels, alpha=0.5)
            logits = self.forward(mix_images).squeeze(1)
            loss = self._criterion(logits, target_a) * lam +                 (1 - lam) * self._criterion(logits, target_b)
        else:
            logits = self.forward(images).squeeze(1)
            loss = self._criterion(logits, labels)
        
        pred = logits.sigmoid().detach().cpu() * 100.
        labels = labels.detach().cpu() * 100.
        
        return loss, pred, labels

class Model_exp054(pl.LightningModule):
    def __init__(self, cfg, val_losses=None):
        super().__init__()
        self.cfg = cfg
        self.__build_model()
        self._criterion = eval(self.cfg.loss)()
        self.transform = get_default_transforms()
        self.save_hyperparameters(cfg)
        self.val_losses = val_losses 

    def __build_model(self):
        self.backbone = self.backbone = PoolFormerModels.poolformer_m36(pretrained=False)
        self.backbone.head = nn.Linear(self.backbone.head.in_features, 128)
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(128+12, 64)
        self.dense2 = nn.Linear(64, 1)

    def forward(self, image, features):
        x = self.get_feature(image)
        x = self.dropout(x)
        x = torch.cat([x, features], dim=1)
        x = self.dense1(x)
        out = self.dense2(x)
        return out

    def get_feature(self, image):
        x = self.backbone(image)
        return x

    def training_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'train')
        self.log("train/loss", loss)
        return {'loss': loss, 'pred': pred, 'labels': labels}
        
    def validation_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'val')
        self.log("val/loss", loss)
        return {'pred': pred, 'labels': labels}
    
    def __share_step(self, batch, mode):
        images, feature, labels = batch
        labels = labels.float() / 100.0
        images = self.transform[mode](images)
        
        if torch.rand(1)[0] < 0.5 and mode == 'train':
            mix_images, target_a, target_b, lam = mixup(images, labels, alpha=self.cfg.mixup.alpha)
            logits = self.forward(mix_images, feature).squeeze(1)
            loss = self._criterion(logits, target_a) * lam + (1 - lam) * self._criterion(logits, target_b)
        else:
            logits = self.forward(images, feature).squeeze(1)
            loss = self._criterion(logits, labels)
        
        pred = logits.sigmoid().detach().cpu() * 100.
        labels = labels.detach().cpu() * 100.
        
        return loss, pred, labels

class Model_exp060(pl.LightningModule):
    def __init__(self, cfg, val_losses=None):
        super().__init__()
        self.cfg = cfg
        self.__build_model()
        self._criterion = eval(self.cfg.loss)()
        self.transform = get_default_transforms()
        self.save_hyperparameters(cfg)
        self.val_losses = val_losses 

    def __build_model(self):
        self.backbone = create_model(
            self.cfg.model.name, pretrained=False, num_classes=128, in_chans=3
        )
        self.dropout = nn.Dropout(0.1)
        self.dense1 = nn.Linear(128+15, 64)
        self.dense2 = nn.Linear(64, 1)

    def forward(self, image, features):
        x = self.get_feature(image)
        x = self.dropout(x)
        x = torch.cat([x, features], dim=1)
        x = self.dense1(x)
        out = self.dense2(x)
        return out

    def get_feature(self, image):
        x = self.backbone(image)
        return x

    def training_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'train')
        self.log("train/loss", loss)
        return {'loss': loss, 'pred': pred, 'labels': labels}
        
    def validation_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'val')
        self.log("val/loss", loss)
        return {'pred': pred, 'labels': labels}
    
    def __share_step(self, batch, mode):
        images, feature, labels = batch
        feature = feature.float()
        labels = labels.float() / 100.0
        images = self.transform[mode](images)
        
        if torch.rand(1)[0] < 0.5 and mode == 'train':
            mix_images, target_a, target_b, lam = mixup(images, labels, alpha=self.cfg.mixup.alpha)
            logits = self.forward(mix_images, feature).squeeze(1)
            loss = self._criterion(logits, target_a) * lam + (1 - lam) * self._criterion(logits, target_b)
        else:
            logits = self.forward(images, feature).squeeze(1)
            loss = self._criterion(logits, labels)
        
        pred = logits.sigmoid().detach().cpu() * 100.
        labels = labels.detach().cpu() * 100.
        
        return loss, pred, labels

class Model_exp065(pl.LightningModule):
    def __init__(self, cfg, val_losses=None):
        super().__init__()
        self.cfg = cfg
        self.__build_model()
        self._criterion = eval(self.cfg.loss)()
        self.transform = get_default_transforms()
        self.save_hyperparameters(cfg)
        self.val_losses = val_losses 

    def __build_model(self):
        self.backbone = create_model(
            self.cfg.model.name, pretrained=False, num_classes=0, in_chans=3
        )
        num_features = self.backbone.num_features
        self.fc = nn.Sequential(
            nn.Dropout(0.5), nn.Linear(num_features, self.cfg.model.output_dim)
        )

    def forward(self, x):
        f = self.backbone(x)
        out = self.fc(f)
        return out

    def training_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'train')
        self.log("train/loss", loss)
        return {'loss': loss, 'pred': pred, 'labels': labels}
        
    def validation_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'val')
        self.log("val/loss", loss)
        return {'pred': pred, 'labels': labels}
    
    def __share_step(self, batch, mode):
        images, feature, labels = batch
        feature = feature.float()
        labels = labels.float() / 100.0
        images = self.transform[mode](images)
        
        if torch.rand(1)[0] < 0.5 and mode == 'train':
            mix_images, target_a, target_b, lam = mixup(images, labels, alpha=self.cfg.mixup.alpha)
            logits = self.forward(mix_images, feature).squeeze(1)
            loss = self._criterion(logits, target_a) * lam + (1 - lam) * self._criterion(logits, target_b)
        else:
            logits = self.forward(images, feature).squeeze(1)
            loss = self._criterion(logits, labels)
        
        pred = logits.sigmoid().detach().cpu() * 100.
        labels = labels.detach().cpu() * 100.
        
        return loss, pred, labels
    
class Model_exp069(pl.LightningModule):
    def __init__(self, cfg, val_losses=None):
        super().__init__()
        self.cfg = cfg
        self.__build_model()
        self._criterion = eval(self.cfg.loss)()
        self.transform = get_default_transforms()
        self.save_hyperparameters(cfg)
        self.val_losses = val_losses 

    def __build_model(self):
        self.backbone = create_model(
            self.cfg.model.name, pretrained=False, num_classes=128, in_chans=3
        )
        self.dropout1 = nn.Dropout(0.5)
        self.dense1 = nn.Linear(128+3, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.dense2 = nn.Linear(64, 1)
        num_features = self.backbone.num_features

    def forward(self, image, features):
        x = self.backbone(image)
        x = self.dropout1(x)
        x = torch.cat([x, features], dim=1)
        x = self.dense1(x)
        x = self.dropout2(x)
        out = self.dense2(x)
        return out

    def training_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'train')
        self.log("train/loss", loss)
        return {'loss': loss, 'pred': pred, 'labels': labels}
        
    def validation_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'val')
        self.log("val/loss", loss)
        return {'pred': pred, 'labels': labels}
    
    def __share_step(self, batch, mode):
        images, feature, labels = batch
        feature = feature.float()
        labels = labels.float() / 100.0
        images = self.transform[mode](images)
        
        if torch.rand(1)[0] < 0.5 and mode == 'train':
            mix_images, target_a, target_b, lam = mixup(images, labels, alpha=self.cfg.mixup.alpha)
            logits = self.forward(mix_images, feature).squeeze(1)
            loss = self._criterion(logits, target_a) * lam + (1 - lam) * self._criterion(logits, target_b)
        else:
            logits = self.forward(images, feature).squeeze(1)
            loss = self._criterion(logits, labels)
        
        pred = logits.sigmoid().detach().cpu() * 100.
        labels = labels.detach().cpu() * 100.
        
        return loss, pred, labels
    
class Model_exp074(pl.LightningModule):
    def __init__(self, cfg, val_losses=None):
        super().__init__()
        self.cfg = cfg
        self.__build_model()
        self._criterion = eval(self.cfg.loss)()
        self.transform = get_default_transforms()
        self.save_hyperparameters(cfg)
        self.val_losses = val_losses 

    def __build_model(self):
        self.backbone = create_model(
            self.cfg.model.name, pretrained=False, num_classes=128, in_chans=3
        )
        self.dropout1 = nn.Dropout(0.5)
        self.dense1 = nn.Linear(128+15, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.dense2 = nn.Linear(64, 1)
        num_features = self.backbone.num_features

    def forward(self, image, features):
        x = self.get_feature(image)
        x = self.dropout1(x)
        x = torch.cat([x, features], dim=1)
        x = self.dense1(x)
        x = self.dropout2(x)
        out = self.dense2(x)
        return out

    def get_feature(self, image):
        x = self.backbone(image)
        return x

    def training_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'train')
        self.log("train/loss", loss)
        return {'loss': loss, 'pred': pred, 'labels': labels}
        
    def validation_step(self, batch, batch_idx):
        loss, pred, labels = self.__share_step(batch, 'val')
        self.log("val/loss", loss)
        return {'pred': pred, 'labels': labels}
    
    def __share_step(self, batch, mode):
        images, feature, labels = batch
        feature = feature.float()
        labels = labels.float() / 100.0
        images = self.transform[mode](images)
        
        if torch.rand(1)[0] < 0.5 and mode == 'train':
            mix_images, target_a, target_b, lam = mixup(images, labels, alpha=self.cfg.mixup.alpha)
            logits = self.forward(mix_images, feature).squeeze(1)
            loss = self._criterion(logits, target_a) * lam + (1 - lam) * self._criterion(logits, target_b)
        else:
            logits = self.forward(images, feature).squeeze(1)
            loss = self._criterion(logits, labels)
        
        pred = logits.sigmoid().detach().cpu() * 100.
        labels = labels.detach().cpu() * 100.
        
        return loss, pred, labels
    
def transform_get_image_shape_describe(input_df): 
    pathes = input_df['Id'].values
    shapes = []
    for path in tqdm(pathes):
        img = Image.open(path)
        height, width = img.height, img.width
        shapes.append([height, width])
    input_df[["height", "width"]] = shapes
    input_df["aspect"] = input_df["height"] / input_df["width"]
    height_mean = 904.2843018563358
    height_std = 156.90598049629264
    width_mean = 804.4262510088781
    width_std = 270.21192072081044
    if len(input_df)==8:#testだと全部同じ大きさで標準化ミスるからエラー除去でいれる。
        input_df['height'] = input_df['height'].fillna(0)
        input_df['width'] = input_df['width'].fillna(0)
    else:
        input_df['height'] = input_df['height'].apply(lambda x: (x-height_mean)/ height_std)
        input_df['width'] = input_df['width'].apply(lambda x: (x-width_mean)/ width_std)

    print(f'height_mean{height_mean}_std{height_std}')
    print(f'width_mean{width_mean}_std{width_std}')
    return input_df

test_df = pd.read_csv(os.path.join(config_exp049.root, "test.csv"))
test = test_df.copy()
test_df["Id"] = test_df["Id"].apply(lambda x: os.path.join(config_exp049.root, "test", x + ".jpg"))
test_df = transform_get_image_shape_describe(test_df)
dataloader_exp049 = PetfinderDataModule(test_df, config_exp049).test_dataloader()
dataloader_exp054 = PetfinderDataModule(test_df, config_exp054).test_dataloader()
dataloader_exp060 = PetfinderDataModule(test_df, config_exp060).test_dataloader()
dataloader_exp065 = PetfinderDataModule(test_df, config_exp065).test_dataloader()
dataloader_exp069 = PetfinderDataModule(test_df, config_exp069).test_dataloader()
dataloader_exp074 = PetfinderDataModule(test_df, config_exp074).test_dataloader()


def get_predict_feature(test_loader, model, device):
    model = model.to(device)
    model = model.eval()
    predicts = []
    for images, features in test_loader:
        images = images.to(device)
        features = features.to(device).float()
        images = get_default_transforms()['val'](images)
        batch_size = images.size(0)
        with torch.no_grad():
            predict = model(images, features).sigmoid().detach().cpu().numpy() * 100
        predicts.append(predict)
    predicts = np.concatenate(predicts)
    return predicts

def get_predict(test_loader, model, device):
    model = model.to(device)
    model = model.eval()
    predicts = []
    for images in test_loader:
        images = images.to(device)
        images = get_default_transforms()['val'](images)
        batch_size = images.size(0)
        with torch.no_grad():
            predict = model(images).sigmoid().detach().cpu().numpy() * 100
        predicts.append(predict)
    predicts = np.concatenate(predicts)
    return predicts

IMG_PREDICTS = []
tmp = []
for fold in tqdm(range(config_exp049.n_splits)):
    model = Model_exp049(config_exp049)
    model_path = "../input/petfinder-334model/exp049"
    model.load_state_dict(torch.load(f'{model_path}/fold{fold}/best_loss_fold{fold}.pth'))
    predicts = get_predict(dataloader_exp049, model, 'cuda')
    tmp.append(predicts)
    
    del model
    torch.cuda.empty_cache()
IMG_PREDICTS.append(tmp)
    
tmp = []
for fold in tqdm(range(config_exp054.n_splits)):
    model = Model_exp054(config_exp054)
    model_path = "../input/petfinder-224model/exp054"
    model.load_state_dict(torch.load(f'{model_path}/fold{fold}/best_loss_fold{fold}.pth'))
    predicts = get_predict_feature(dataloader_exp054, model, 'cuda')
    tmp.append(predicts)
    
    del model
    torch.cuda.empty_cache()
IMG_PREDICTS.append(tmp)

tmp = []
for fold in tqdm(range(config_exp060.n_splits)):
    model = Model_exp060(config_exp060)
    model_path = "../input/petfinder-224model/exp060"
    model.load_state_dict(torch.load(f'{model_path}/fold{fold}/best_loss_fold{fold}.pth'))
    predicts = get_predict_feature(dataloader_exp060, model, 'cuda')
    tmp.append(predicts)
    
    del model
    torch.cuda.empty_cache()
IMG_PREDICTS.append(tmp)

tmp = []
for fold in tqdm(range(config_exp065.n_splits)):
    model = Model_exp065(config_exp065)
    model_path = "../input/petfinder-334model/exp065"
    model.load_state_dict(torch.load(f'{model_path}/fold{fold}/best_loss_fold{fold}.pth'))
    predicts = get_predict(dataloader_exp065, model, 'cuda')
    tmp.append(predicts)
    
    del model
    torch.cuda.empty_cache()
IMG_PREDICTS.append(tmp)

tmp = []
for fold in tqdm(range(config_exp069.n_splits)):
    model = Model_exp069(config_exp069)
    model_path = "../input/petfinder-224model/exp069"
    model.load_state_dict(torch.load(f'{model_path}/fold{fold}/best_loss_fold{fold}.pth'))
    predicts = get_predict_feature(dataloader_exp069, model, 'cuda')
    tmp.append(predicts)
    
    del model
    torch.cuda.empty_cache()
IMG_PREDICTS.append(tmp)

tmp = []
for fold in tqdm(range(config_exp074.n_splits)):
    model = Model_exp074(config_exp074)
    model_path = "../input/petfinder-224model/exp074"
    model.load_state_dict(torch.load(f'{model_path}/fold{fold}/best_loss_fold{fold}.pth'))
    predicts = get_predict_feature(dataloader_exp074, model, 'cuda')
    tmp.append(predicts)
    
    del model
    torch.cuda.empty_cache()
IMG_PREDICTS.append(tmp)

def load_ensemble_model(path):
    with open(path, mode='rb') as fp:
        clf = pickle.load(fp)
    return clf

ensemble_predict = []
ensemble_model_exp_path = '../input/d/ktakita/petfinder-ensumble/exp085_ensumble'
for fold in range(5):
    X = np.concatenate([IMG_PREDICTS[0][fold],
                        IMG_PREDICTS[1][fold],
                        IMG_PREDICTS[2][fold],
                        IMG_PREDICTS[3][fold],
                        IMG_PREDICTS[4][fold],
                        IMG_PREDICTS[5][fold]],  
                       axis=1)
    ensemble_model_path = f'{ensemble_model_exp_path}/fold{fold}_{config_ensemble.model}.pickle'
    model = load_ensemble_model(ensemble_model_path)
    pred = model.predict(X)
    ensemble_predict.append(pred)
    
pd.DataFrame(ensemble_predict)

predict = np.mean(ensemble_predict, 0)

predictions = predict
test['Pawpularity'] = predictions
test[['Id', 'Pawpularity']].to_csv('submission_at.csv', index=False)
#display(test[['Id', 'Pawpularity']].head(8))


If for semantic segmentation, please install mmsegmentation first
If for detection, please install mmdetection first
1.4.4
height_mean904.2843018563358_std156.90598049629264
width_mean804.4262510088781_std270.21192072081044


100%|██████████| 5/5 [00:04<00:00,  1.13it/s]


In [2]:
%%python

import os
import sys
import warnings
from pprint import pprint
from glob import glob
from tqdm.notebook import tqdm
import pickle
import glob

from sklearn.linear_model import Ridge
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torchvision.transforms as T
sys.path.append('../input/pythonbox')
from box import Box
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model
from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
#from pytorch_grad_cam import GradCAMPlusPlus
#from pytorch_grad_cam.utils.image import show_cam_on_image
from PIL import Image
from fastai.vision.all import *

import pytorch_lightning as pl
from pytorch_lightning.utilities.seed import seed_everything
from pytorch_lightning import callbacks
from pytorch_lightning.callbacks.progress import ProgressBarBase
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning import LightningDataModule, LightningModule

#sys.path.append('../input/poolformer-master')
#import models as PoolFormerModels #poolformer

print(pl.__version__)
warnings.filterwarnings("ignore")
IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB

sys.path.append(f'../input/petfinder-scripts')


class PetfinderDataModule(LightningDataModule):
    def __init__(
        self,
        test_df,
        dataset,
        cfg,
    ):
        super().__init__()
        self._test_df = test_df
        self._dataset = dataset
        self._cfg = cfg

    def __create_dataset(self, train=True):
        return (
            self._dataset(self._test_df, self._cfg.transform.image_size)
            if train
            else self._dataset(self._test_df, self._cfg.transform.image_size)
        )

    def test_dataloader(self):
        dataset = self.__create_dataset(True)
        self._cfg.val_loader.batch_size = 32
        self._cfg.val_loader.num_workers = os.cpu_count()
        return DataLoader(dataset, **self._cfg.val_loader)

def transform_get_image_shape_describe(input_df): 
    pathes = input_df['Id'].values
    shapes = []
    for path in tqdm(pathes):
        img = Image.open(path)
        height, width = img.height, img.width
        shapes.append([height, width])
    input_df[["height", "width"]] = shapes
    input_df["aspect"] = input_df["height"] / input_df["width"]
    height_mean = 904.2843018563358
    height_std = 156.90598049629264
    width_mean = 804.4262510088781
    width_std = 270.21192072081044
    if len(test_df)==8:#testだと全部同じ大きさで標準化ミスるからエラー除去でいれる。
        input_df['height'] = input_df['height'].fillna(0)
        input_df['width'] = input_df['width'].fillna(0)
    else:
        input_df['height'] = input_df['height'].apply(lambda x: (x-height_mean)/ height_std)
        input_df['width'] = input_df['width'].apply(lambda x: (x-width_mean)/ width_std)

    print(f'height_mean{height_mean}_std{height_std}')
    print(f'width_mean{width_mean}_std{width_std}')
    return input_df
    
def get_predict_feature(model, test_loader, transforms,device):
    model = model.to(device)
    model = model.eval()
    predicts = []
    for images, features in test_loader:
        images = images.to(device)
        features = features.to(device).float()
        images = transforms()['val'](images)
        batch_size = images.size(0)
        with torch.no_grad():
            predict = model(images, features).sigmoid().detach().cpu().numpy() * 100
        predicts.append(predict)
    predicts = np.concatenate(predicts)
    return predicts

def get_predict(model, test_loader, transforms,device):
    model = model.to(device)
    model = model.eval()
    predicts = []
    for images in test_loader:
        images = images.to(device)
        images = transforms()['val'](images)
        batch_size = images.size(0)
        with torch.no_grad():
            predict = model(images).sigmoid().detach().cpu().numpy() * 100
        predicts.append(predict)
    predicts = np.concatenate(predicts)
    return predicts

def model_load(model, exp_name, model_path):
    if exp_name == 'exp092' or exp_name == 'exp100':
        model.load_state_dict(torch.load(model_path)['state_dict'])
    else:
        model.load_state_dict(torch.load(model_path))
    return model

import exp108

test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
test_df["Id"] = test_df["Id"].apply(lambda x: os.path.join('../input/petfinder-pawpularity-score', "test", x + ".jpg"))
test_df = transform_get_image_shape_describe(test_df)

IMG_PREDICTS = []
exp_name = 'exp108'
print(exp_name)
config = eval(exp_name).config
config.model.pretrain = False
dataloader = PetfinderDataModule(test_df, 
                                 eval(exp_name).PetfinderDataset, 
                                 config).test_dataloader()

tmp = []
for fold in tqdm(range(config.n_splits)):
    model = eval(exp_name).Model(config)
    model_path = glob.glob(os.path.join('../input/petfinder-exp108', f'{exp_name}/fold{fold}/best_loss_fold{fold}.*'))[0]
    model.load_state_dict(torch.load(model_path)['state_dict'])
    predicts = get_predict_feature(model, dataloader, eval(exp_name).get_default_transforms, 'cuda')

    tmp.append(predicts)

    del model
    torch.cuda.empty_cache()
IMG_PREDICTS = tmp

#predict make
preds = []
fold_num = len(IMG_PREDICTS)
for fold in range(fold_num):
    preds.append(IMG_PREDICTS[fold] / fold_num)
predict = np.sum(preds, 0)

predictions = predict

test = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
test['Pawpularity'] = predictions
test[['Id', 'Pawpularity']].to_csv('submission_exp108.csv', index=False)
#display(test[['Id', 'Pawpularity']].head(8))

1.4.4
  0%|          | 0/8 [00:00<?, ?it/s]
height_mean904.2843018563358_std156.90598049629264
width_mean804.4262510088781_std270.21192072081044
exp108
  0%|          | 0/10 [00:00<?, ?it/s]


In [3]:
%%python
import sys
sys.path.append('/kaggle/input/timm-pytorch-image-models/pytorch-image-models-master')

# Python Libraries
import os
import gc

# Third party
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# Pytorch 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

# Pytorch Image Models
from timm import create_model
from torchvision.io import read_image
import torchvision.transforms as T

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ====================================================
# Dataset
# ====================================================
class PetfinderDataset(Dataset):
    def __init__(self, df, feature_cols, image_size=224):
        self._X = df["Id"].values
        self.meta = df[feature_cols].values
        self._y = None
#         if "Pawpularity" in df.keys():
#             self._y = df["Pawpularity"].values
        self._transform = T.Compose([
                                        T.Resize(image_size),  # 1
                                        T.CenterCrop([image_size, image_size]),  # 2
                                    ]
                                    )

    def __len__(self):
        return len(self._X)

    def __getitem__(self, idx):
        image_path = self._X[idx]
        image = read_image(image_path)
        image = self._transform(image)
        features = torch.FloatTensor(self.meta[idx, :])
        
        return image, features 
# ====================================================
# Transforms
# ====================================================
IMAGENET_MEAN = [0.485, 0.456, 0.406]  # RGB
IMAGENET_STD = [0.229, 0.224, 0.225]  # RGB
def get_default_transforms():
    transform = {
        "val": T.Compose(
            [
                T.ConvertImageDtype(torch.float),
                T.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
            ]
        ),
    }
    return transform   
# ====================================================
# model
# ====================================================
class CustomModel(nn.Module):
    def __init__(self, cfg, pretrained=False):
        super().__init__()
        self.cfg = cfg
        self.backbone = create_model(model_name=self.cfg.model_name,
                                     pretrained=pretrained,
                                     in_chans=self.cfg.in_chans,
                                     num_classes=0)
        self.dropout1 = nn.Dropout(p=0.5)
        self.dropout2 = nn.Dropout(p=0.5)
        self.fc = nn.Linear(3084, self.cfg.target_size)
        
#         self.fc = nn.Sequential(
#             nn.Dropout(0.5),
#             nn.LazyLinear(self.cfg.target_size)
#         )
    def get_feature(self, x, features):
        return self.backbone(x)
        
    def forward(self, x, features):
        f = self.backbone(x) # (bs, embedding_size)
        f = self.dropout1(f)
        if features.shape[1] != 0:
            f = torch.cat([f, features],dim=1)
            f =  self.dropout2(f)
        out = self.fc(f)
        return out
    
def get_model(cfg):
    model = CustomModel(cfg, pretrained=False)
    return model

def predict(model, loader):
    model = model.to(device)
    model = model.eval()
    predicts = []
    for batch in tqdm(loader):
        images, features = batch
        images = images.to(device)
        features = features.to(device)
        images = get_default_transforms()['val'](images)
        batch_size = images.size(0)
        with torch.no_grad():
            predict = model(images, features).sigmoid().detach().cpu().numpy() * 100
        predicts.append(predict)
    predicts = np.concatenate(predicts)
    return predicts

# ====================================================
# Config
# ====================================================
class CFG_029:
    debug = False
    model_name = 'dm_nfnet_f3'
    img_size = 512
    in_chans = 3
    n_fold = 5
    target_size = 1
    # DataLoader
    loader = {
        "valid": {
            "batch_size": 32,
            "num_workers": 4,
            "shuffle": False,
            "pin_memory": True,
            "drop_last": False
        }
    }
    # exp020
    feature_cols = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory',
        'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur',
        ]

df_test = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
df_test["Id"] = df_test["Id"].apply(lambda x: os.path.join('../input/petfinder-pawpularity-score', "test", x + ".jpg"))
    
loader = DataLoader(PetfinderDataset(df_test, CFG_029.feature_cols, CFG_029.img_size),**CFG_029.loader['valid'])

tmp = []
for i in tqdm(range(CFG_029.n_fold)):
    model = get_model(CFG_029)
    model.load_state_dict(torch.load(f'../input/pet2teyomodel1/exp029_1759169/exp029_fold{i}_best.pth', map_location=device))
    predicts = predict(model, loader)
    tmp.append(predicts)
    del model
    torch.cuda.empty_cache()
    gc.collect()

test = pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
test['Pawpularity'] = np.mean(tmp,axis=0)
test[['Id', 'Pawpularity']].to_csv('submission_exp029.csv',index=False)

  0%|          | 0/1 [00:00<?, ?it/s]


In [4]:
%%python
#import pandas as pd
#if pd.read_csv('../input/petfinder-pawpularity-score/test.csv').shape[0] == 8:
#    exit()
    
import sys
import gc
from tqdm.auto import tqdm
sys.path = ["../input/pytorch-1-10-1/"] + sys.path
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model

from fastai.vision.all import *
print(torch.__version__)

set_seed(999, reproducible=True)
BATCH_SIZE = 16

dataset_path = Path('../input/petfinder-pawpularity-score/')

seed = 999
set_seed(seed, reproducible=True)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.use_deterministic_algorithms = True

def petfinder_rmse(input,target):
    return 100*torch.sqrt(F.mse_loss(F.sigmoid(input.flatten()), target))

test_df = pd.read_csv(dataset_path/'test.csv')
test_df.head()

test_df['Pawpularity'] = [1]*len(test_df)
test_df['path'] = test_df['Id'].map(lambda x:str(dataset_path/'test'/x)+'.jpg')
test_df = test_df.drop(columns=['Id'])
test_df['norm_score'] = test_df['Pawpularity'] / 100

def get_data(img_size):
    dls = ImageDataLoaders.from_df(test_df, #pass in train DataFrame
                               #valid_col='is_valid', #
                               seed=999, #seed
                               fn_col='path', #filename/path is in the second column of the DataFrame
                               label_col='norm_score', #label is in the first column of the DataFrame
                               y_block=RegressionBlock, #The type of target
                               bs=BATCH_SIZE, #pass in batch size
                               num_workers=8,
                               item_tfms=Resize(img_size), #pass in item_tfms
                               batch_tfms=setup_aug_tfms([Brightness(), Contrast(), Hue(), Saturation()])) #pass in batch_tfms
    
    return dls



model_weights = {'exp7_cait_m36_384': 0.10573687177566875,
 'exp4_xcit_small_24_p16_384_dist': 0.09017084075938603,
 'exp4_crossvit_18_dagger_408': 0.0864555998708494,
 'exp4_xcit_small_24_p8_384_dist': 0.08478909740799544,
 'exp15_vit_base_patch16_224_miil_in21k': 0.08388133976905879,
 'exp7_swin_large_patch4_window12_384_in22k': 0.08162600824050978,
 'exp9_cait_m36_384': 0.07951668818830163,
 'exp7_vit_base_patch16_224_miil_in21k': 0.07448271317999128,
 'exp7_swin_base_patch4_window12_384': 0.07014647726439097,
 'exp7_jx_nest_base': 0.06841255969223342,
 'exp8_vit_base_patch16_224_miil_in21k': 0.06359775694462852,
 'exp7_vit_base_r50_s16_384': 0.061862705948120524,
 'exp15_vit_large_patch16_224': 0.0546458052004623}


N_FOLDS = 5
all_preds = []

for path, w in tqdm(model_weights.items()):
    model_name = path.split('_', 1)[-1]
    try:
        img_size = int(model_name.split('_')[-1])
    except:
        try:
            img_size = int(model_name.split('_')[-2])
        except:
            img_size = 224
    dls = get_data(img_size)
    
    def proc(pred):
        return pred
    num_classes = 1
    if 'exp4_' in path:
        model_dir = '../input/model-exp4-0106/pet'
        loss = BCEWithLogitsLossFlat()
    elif 'exp7_' in path:
        model_dir = '../input/model-exp7-0106/pet'
        loss = BCEWithLogitsLossFlat()
    elif 'exp8_' in path:
        model_dir = '../input/model-exp8-9-15-0106/pet'
        loss = MSELossFlat()
    elif 'exp9_' in path:
        model_dir = '../input/model-exp8-9-15-0106/pet' 
        loss = CrossEntropyLossFlat()
        num_classes = 100
        def proc(pred):
            return (pred.argmax(axis=1) + 1) / 100
    elif 'exp15_' in path:
        model_dir = '../input/model-exp8-9-15-0106/pet'
        loss = MSELossFlat()
        def proc(pred):
            return (np.exp(pred) / 100).clip(0.01, 1)
    else:
        raise
            
    for i in range(N_FOLDS):
        model = create_model(model_name, pretrained=False, num_classes=num_classes)
        model.load_state_dict(torch.load(f'{model_dir}/{path}/{model_name}_{i}.pth'))

        learn = Learner(dls, model, 
                        loss_func=loss,
                        metrics=petfinder_rmse).to_fp16()

        test_dl = dls.test_dl(test_df)

        preds, _ = learn.get_preds(dl=test_dl)
        #preds, _ = learn.tta(dl=test_dl, n=2, beta=0)
        
        preds = proc(preds).flatten()
        
        all_preds.append(preds * w / N_FOLDS)

        del learn
        torch.cuda.empty_cache()
        gc.collect()
    del dls
    torch.cuda.empty_cache()
    gc.collect()

sample_df = pd.read_csv(dataset_path/'sample_submission.csv')
preds = np.sum(np.stack(all_preds), axis=0)
sample_df['Pawpularity'] = preds*100
sample_df.to_csv('submission_tkm.csv',index=False)

1.10.0+cu113


 38%|███▊      | 5/13 [03:03<03:46, 28.34s/it]../input/pytorch-1-10-1/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
100%|██████████| 13/13 [09:27<00:00, 43.64s/it]


In [5]:
import pandas as pd

sub_at = pd.read_csv('submission_at.csv', index_col='Id')
sub_tkm = pd.read_csv('submission_tkm.csv', index_col='Id')
sub_exp108 = pd.read_csv('submission_exp108.csv', index_col='Id')
sub_exp029 = pd.read_csv('submission_exp029.csv', index_col='Id')

#[0.502754   0.18612928 0.31658946]
#[0.51461806 0.16874229 0.3218239 ]
#['exp108', 'exp029', 'oof0(tkmsan)', 'oof1(at)']
#[0.29673496 0.17641786 0.41963073 0.11312639]
sub = (0.29673496 * sub_exp108 + 0.17641786 * sub_exp029 + 0.41963073 * sub_tkm + 0.11312639 * sub_at)

sub.to_csv('submission_ens.csv')
sub


,Pawpularity
Id,
4128bae22183829d2b5fea10effdb0c3,49.200574
43a2262d7738e3d420d453815151079e,49.188477
4e429cead1848a298432a0acad014c9d,49.115084
80bc3ccafcc51b66303c2c263aa38486,49.370076
8f49844c382931444e68dffbe20228f4,49.026995
b03f7041962238a7c9d6537e22f9b017,49.265595
c978013571258ed6d4637f6e8cc9d6a3,49.417783
e0de453c1bffc20c22b072b34b54e50f,49.135504


In [6]:
import os
import pickle
import numpy as np
from PIL import Image
import imagehash
from tqdm.auto import tqdm

DEBUG = pd.read_csv('../input/petfinder-pawpularity-score/test.csv').shape[0] == 8

df1 = pd.read_csv('../input/pet-data/df1_train_test_v5.csv')

if DEBUG:
    df2 = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
    df2['pred'] = 0.5
    df2['path'] = df2['Id'].map(lambda x:str(f'../input/petfinder-pawpularity-score/train/{x}.jpg'))
else:
    df2 = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
    df2['pred'] = df2['Id'].map(pd.read_csv('submission_ens.csv', index_col='Id')['Pawpularity']) / 100
    df2['path'] = df2['Id'].map(lambda x:str(f'../input/petfinder-pawpularity-score/test/{x}.jpg'))
    
df2['hash'] = [imagehash.average_hash(Image.open(x))
                    for x in tqdm(df2['path'].values)]

  0%|          | 0/9912 [00:00<?, ?it/s]

In [7]:
df1['hash'] = df1['hash'].astype(str)
df2['hash'] = df2['hash'].astype(str)
df = pd.merge(df2, df1, how='left', on='hash')

In [8]:
if df.shape[0] > 0:
    with open('../input/0113-2nd-stage-gbdt-fixed-w-newmodels3/list_model_lgb_last_comp_all.pkl', 'rb') as f:
        models = pickle.load(f)
    df['len_desc'] = df['Description'].fillna('').map(len)
    df['Name'] = df['Name'].fillna('').replace('No Name Yet', '')
    df['len_name'] = df['Name'].map(len)
    
    pred = 0
    for model in models:
        pred += model.predict(df[model.feature_name()])
    pred /= len(models)
    
    df_pred = pd.DataFrame()
    df_pred['Id'] = df['Id'].values
    df_pred['Pawpularity'] = (pred * 100).clip(1, 100)
    
    sub = df_pred.groupby('Id')[['Pawpularity']].mean()
    #sub = pd.read_csv('submission_ens.csv')
    #sub = sub[~sub['Id'].isin(df_pred['Id'])].append(df_pred)
    #sub = sub.set_index('Id').sort_index()
    #sub.to_csv('submission.csv')
else:
    sub = pd.read_csv('submission_ens.csv', index_col='Id')
    #sub.to_csv('submission.csv', index=False)
    

pred = sub['Pawpularity'].values / 100
#mu = pred.mean()
#pred = (pred - mu) + 0.38362035973750004
pred = pred.clip(0.01, 1)

sub['Pawpularity'] = pred * 100
sub.to_csv('submission.csv')
sub

,Pawpularity
Id,
0007de18844b0dbbb5e1f607da0606e0,60.485896
0009c66b9439883ba2750fb825e1d7db,45.796748
0013fd999caf9a3efe1352ca1b0d937e,56.013132
0018df346ac9c1d8413cfcc888ca8246,51.530062
001dc955e10590d3ca4673f034feeef2,51.243319
...,...
ffbfa0383c34dc513c95560d6e1fdb57,49.588228
ffcc8532d76436fc79e50eb2e5238e45,51.530062
ffdf2e8673a1da6fb80342fa3b119a20,51.530062
